In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd 
dataset_df=pd.read_excel('/kaggle/input/parphrase/para.xlsx')

In [4]:
!pip install simpletransformers datasets tqdm pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 42.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 78.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 63.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=b5f20f99c88e0cf132e1d54c400715bd12f67092869f3a25c2bf92ca25024030
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.

In [5]:

dataset_df.columns = ["input_text","target_text"]
dataset_df["prefix"] = "paraphrase"

In [7]:
from simpletransformers.t5 import T5Model
from sklearn.model_selection import train_test_split
import sklearn

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [8]:
train_data,test_data = train_test_split(dataset_df,test_size=0.1)


In [9]:
train_data

,input_text,target_text,prefix
30,Elle a préparé une délicieuse tarte aux fraises.,Une tarte succulente aux fraises a été confect...,paraphrase
1063,Elle parle couramment trois langues.,Sa maîtrise linguistique s'étend à trois langues.,paraphrase
1900,Elle parle couramment trois langues étrangères.,Elle maîtrise parfaitement trois langues étran...,paraphrase
264,Je vais faire du shopping cet après-midi.,"Cet après-midi, je vais faire des achats.",paraphrase
1818,J'aime prendre une longue promenade en bord de...,Se balader le long de la mer est une activité ...,paraphrase
...,...,...,...
1018,Elle prépare un délicieux repas.,Elle s'affaire à concocter un repas succulent.,paraphrase
71,Nous avons gagné le match de football.,Nous sommes sortis victorieux du match de foot...,paraphrase
28,Il est important de prendre soin de notre plan...,La préservation de notre planète revêt une imp...,paraphrase
1874,Elle a remporté la compétition de danse.,La victoire dans la compétition de danse lui r...,paraphrase


In [10]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": False,
    'adam_epsilon': 1e-08,
    'eval_batch_size': 6,
    'fp_16': False,

    'gradient_accumulation_steps': 16,
    'learning_rate': 0.0003,
    'max_grad_norm': 1.0,
    'n_gpu': 1,
    'seed': 42,
    'train_batch_size': 6,
    'warmup_steps': 0,
    'weight_decay': 0.0
}

In [11]:
model = T5Model("t5","t5-small", args=args,use_cuda=False)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
model.train_model(train_data, eval_data=test_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)


  0%|          | 0/1809 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/302 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/302 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/302 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/302 [00:00<?, ?it/s]

(72, 1.2778723215322114)

In [13]:
from simpletransformers.t5 import T5Model
from pprint import pprint
import os

In [14]:

root_dir = os.getcwd()
trained_model_path = os.path.join(root_dir,"outputs")
     

In [15]:
args = {
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 5,
    "num_beams":1
}

In [16]:
trained_model = T5Model("t5",trained_model_path,args=args,use_cuda=False)


In [18]:
prefix = "paraphrase"

pred = trained_model.predict([f"{prefix}:j'aime travailler avec des models d'intelligence artificielle ."])
for i in pred :
   print(i)


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/5 [00:00<?, ?it/s]

["Je suis encouragé à travailler avec des modèles d'intelligence artificielle.", "Ayant hâte de travailler avec des modèles d'intelligence artificielle, j'apprécie travailler.", "J'apprécie travailler avec des modèles d'intelligence artificielle.", "Avec des modèles d'intelligence artificielle, j'apprécie travailler.", "J'apprécie travailler avec des modèles d'intelligence artificielle."]
